# PELIC spelling

This notebook adds further processing to `PELIC_compiled.csv`  in the [`PELIC-dataset`](https://github.com/ELI-Data-Mining-Group/PELIC-dataset) repo by creating a column of tok_POS whose spelling has been automatically corrected.

**Notebook contents:**
- [Building `non_words_df`](#Building-non_words_df)
- [Building `misspell_df`](#Building-misspell_df)
- [Possible segmentation](#Applying-segmentation)
- [Applying spelling correction](#Applying-spelling-correction)
- [Incorporating corrections into `pelic_df`](#Incorporating-corrections-into-pelic_df)

## Building non_words_df
In this section, we build a dataframe, `non_words_df`, which collects all of the non-words from the PELIC dataset (in `PELIC_compiled.csv`). The final dataframe has the following columns:
- `non_word`: tuples with the non-words and their parts of speech
- `sentence`: the complete sentence containing the non-word to provide context
- `answer_id`: the id of the text they come from

In [1]:
# Import necessary modules

import pandas as pd
import pprint
import numpy as np
from ast import literal_eval
import nltk
import random
from pelitk import lex
import string
import re

In [2]:
# Read in PELIC_compiled.csv

pelic_df = pd.read_csv("../PELIC-dataset/PELIC_compiled.csv", index_col = 'answer_id', # answer_id is unique
                      dtype = {'level_id':'object','question_id':'object','version':'object','course_id':'object'}, # str not ints
                               converters={'tokens':literal_eval,'tok_lem_POS':literal_eval}) # read in as lists
pelic_df.head()

,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text_len,text,tokens,tok_lem_POS
answer_id,,,,,,,,,,,,
1,eq0,Arabic,Male,149,4,g,5,1,177,I met my friend Nife while I was studying in a...,"[I, met, my, friend, Nife, while, I, was, stud...","((I, i, PRP), (met, meet, VBD), (my, my, PRP$)..."
2,am8,Thai,Female,149,4,g,5,1,137,"Ten years ago, I met a women on the train betw...","[Ten, years, ago, ,, I, met, a, women, on, the...","((Ten, ten, CD), (years, year, NNS), (ago, ago..."
3,dk5,Turkish,Female,115,4,w,12,1,64,In my country we usually don't use tea bags. F...,"[In, my, country, we, usually, do, n't, use, t...","((In, in, IN), (my, my, PRP$), (country, count..."
4,dk5,Turkish,Female,115,4,w,13,1,6,I organized the instructions by time.,"[I, organized, the, instructions, by, time, .]","((I, i, PRP), (organized, organize, VBD), (the..."
5,ad1,Korean,Female,115,4,w,12,1,59,"First, prepare a port, loose tea, and cup.\nSe...","[First, ,, prepare, a, port, ,, loose, tea, ,,...","((First, first, RB), (,, ,, ,), (prepare, prep..."


The focus here is the `tok_lem_POS` column, but all columns will be kept as the entire df will be written out at the end of the notebook.

In [3]:
# Creating small dataframe to be used for finding non-words

non_words = pelic_df[['text','tok_lem_POS']]

**Note:** For spelling correction, it is necessary to decide what list of words will be used for determining if a word is real or not.

Here, we use the [`SCOWL_condensed.txt`](https://github.com/ELI-Data-Mining-Group/PELIC-spelling/blob/master/SCOWL_condensed.txt) file which is a combination of wordlists available for download at http://wordlist.aspell.net/. We include items from all the dictionaries _except_ the abbreviations dictionary. For a detailed look at the compilation of this dictionary, please see the [SCOWL_wordlist](https://github.com/ELI-Data-Mining-Group/PELIC-spelling/blob/master/SCOWL_wordlist.ipynb) notebook.

In [4]:
#Reading in SCOWL_condensed as a set as a lookup list for spelling (500k words)

scowl = set(open("SCOWL_condensed.txt", "r").read().split('\n'))
print(random.sample(scowl,5))

['orthopath', 'plumpened', 'shadfly', 'Burkle', 'unfencing']


In [5]:
scowl = set([x.lower() for x in scowl])
len(scowl)

497552

The following is a list of words which should be considered words but which were previously being labelled as non-words. These items have been manually added to this list based on output later in this notebook. Most of these items are food items, names, or abbreviations.

In [6]:
scowl_supp = open("SCOWL_supp.txt", "r").read().split(',')
scowl_supp = [x[2:-1] for x in scowl_supp]
print(len(scowl_supp))
print(scowl_supp)

241
['adha', 'adj', 'ahamed', 'alaikum', 'anonurlpage', 'antiretroviral', 'arpa', 'atm', 'ave', 'beyonce', 'bibimbap', 'bio', 'biodiesel', 'bioethanol', 'bulgogi', 'bundang', 'cafe', 'carnaval', 'cds', 'cf', 'co', 'comscore', 'cyber', 'ddukboggi', 'def', 'dr', 'eg', 'eid', 'electrospray', 'entrees', 'erectus', 'etc', 'fiance', 'fiancee', 'fiter', 'fitir', 'fitr', 'fl', 'freediving', 'fukubukuro', 'geolinguist', 'hikikomori', 'hp', 'ibt', 'icu', 'iq', 'ipods', 'iriver', 'jetta', 'jul', 'kabsa', 'kaled', 'kawader', 'kennywood', 'km', 'leisureville', 'll', 'maamool', 'mayumi', 'mcdonalds', 'min', 'mongongo', 'nc', 'neuro', 'nian', 'notting', 'okroshka', 'onsen', 'pajeon', 'pbt', 'pc', 'pcs', 'pp', 'pudim', 'puket', 'samear', 'shui', 'sq', 'st', 'staycation', 'sth', 'taoyuan', 'toefl', 'trans', 'transgene', 'tv', 'unsub', 'va', 'vol', 'vs', 'webaholic', 'webaholics', 'webaholism', 'wenjing', 'woong', 'yaoming', 'ying', 'yingdong', 'yugong', 'yuval', 'zi', 'abha', 'achuar', 'ae', 'afandi', 

In [7]:
# Lower case all toks

non_words.tok_lem_POS = non_words.tok_lem_POS.apply(lambda row: [(x[0].lower(),x[1],x[2]) for x in row])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
# Function to find non-words

def spell_check(tok_lem_POS_list):
    word_list = scowl # Choose word_list here. Default is scowl described above.
    not_in_word_list = []
    for tok_lem_POS in tok_lem_POS_list:
        if tok_lem_POS[0] not in word_list and tok_lem_POS[0] not in scowl_supp:
            not_in_word_list.append(tok_lem_POS)
    return not_in_word_list

In [9]:
# Apply spell check function to find all misspelled-words. 

non_words['misspelled_words'] = non_words.tok_lem_POS.apply(spell_check)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
non_words.head()

,text,tok_lem_POS,misspelled_words
answer_id,,,
1,I met my friend Nife while I was studying in a...,"[(i, i, PRP), (met, meet, VBD), (my, my, PRP$)...","[(., ., .), (., ., .), (., ., .), (;, ;, :), (..."
2,"Ten years ago, I met a women on the train betw...","[(ten, ten, CD), (years, year, NNS), (ago, ago...","[(,, ,, ,), (,, ,, ,), (., ., .), (;, ;, :), (..."
3,In my country we usually don't use tea bags. F...,"[(in, in, IN), (my, my, PRP$), (country, count...","[(., ., .), (,, ,, ,), (., ., .), (., ., .), (..."
4,I organized the instructions by time.,"[(i, i, PRP), (organized, organize, VBD), (the...","[(., ., .)]"
5,"First, prepare a port, loose tea, and cup.\nSe...","[(first, first, RB), (,, ,, ,), (prepare, prep...","[(,, ,, ,), (,, ,, ,), (,, ,, ,), (., ., .), (..."


#### Adding context to the dataframe
Seeing the mistakes in the context of a sentence will allow for better manual checking if required.

In [11]:
# Sent-tokenizing the text

non_words['sentence'] = non_words['text'].apply(lambda x: nltk.sent_tokenize(x))

# And delete text column which is no longer needed

del non_words['text']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Checking for hyphenated words tagged as misspellings because SCOWL doesn't contain hypenated words

hyphenated = set([x[0] for x in [x for y in non_words.misspelled_words.to_list() for x in y] if '-' in x[0]])
print(len(hyphenated))
print(list(hyphenated)[:10])

# These need to be removed from the non-words dataframe if composed of valid words

1182
['get-together', 'worn-out', 'ten-year', 'left-handers', 'better-paid', 'no-go', 'full-length', 'check-out', 'growing-up', 'short-lived']


In [13]:
# Hyphenated items whose components are not in scowl - possible misspellings or punctuation strings

sorted([y for y in [x.split('-') for x in hyphenated] if y[0] not in scowl or y[1] not in scowl])

[['', "'"],
 ['', '***', '****'],
 ['', '+'],
 ['', '.'],
 ['',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  ''],
 ["'", ''],
 ['.', ''],
 ['/', ''],
 ['\\\\', ''],
 ['^', '^'],
 ['al', 'qaida'],
 ['austro', 'hungarian'],
 ['cd', 'rom'],
 ['co', 'authored'],
 ['co', 'ed'],
 ['co', 'educational'],
 ['co', 'exist'],
 ['co', 'existence'],
 ['co', 'founded'],
 ['co', 'founder'],
 ['co', 'founders'],
 ['co', 'host'],
 ['co', 'op'],
 ['co', 'operate'],
 ['co', 'operation'],
 ['co', 'pay'],
 ['co', 'pilot'],
 ['co', 'sleeping'],
 ['co', 'star'],
 ['co', 'worker'],
 ['co', 'workers'],
 ['co', 'written'],
 ['co', 'wrote'],
 ['mah', 'jong'],
 ['mid', '80s'],
 ['pay', 'tv'],
 ['roly', 'poly'],
 ['socio', 'cultural'],
 ['socio', 'economic'],
 ['trans', 'fat'],
 ['vis', 'a', 'vis'],
 ['wal', 'mart']]

After manual checking, all the hypenated words are punctuation, real words (or true productive use of affixes) and can be removed from the non-words df.

The following two cells 
1. remove all the hypenated words from the dataframe
2. remove all words that don't contain a letter

However, as all hyphenated word are fine, we will instead just eliminate all words that are not purely composed of letters. This will have the effect of removing the following categories from the dataframe:
- punctuation
- hyphenated words (e.g. well-known)
- contractions (e.g. 'll, 've)
- years (e.g. 1950s)
- ordinals (e.g. 1st, 2nd)

## Tangent
The next three cells use the above information to create a more complete `PELIC-SCOWL.txt` wordlist for use with PELIC data.

In [14]:
%%capture

# Creating a text file of hyphenated list for use elsewhere in creating an PELIC-SCOWL wordlist

hyphenated = {word for word in hyphenated if any(x.isalpha() for x in word)}
with open('hyphens.txt', 'w') as f:
    for item in hyphenated:
        f.write("%s\n" % item)

In [15]:
%%capture

# Creating a text file of contractions for use elsewhere in creating an PELIC-SCOWL wordlist

contractions = {"'ll","'ve","n't","'m","'s","'d","'re","'ve"}
with open('contractions.txt', 'w') as f:
    for item in contractions:
        f.write("%s\n" % item)

In [16]:
# Reading in 'all_names' file

all_names = set(open("all_names.txt", "r").read().split('\n'))
all_names = {x.capitalize() for x in all_names}
print(random.sample(all_names,5))

['Darm', 'Staggers', 'Fryer', 'Bertuzzi', 'Goade']


In [17]:
%%capture

# Combining SCOWL_condensed, hyphenated, and contraction lists

pelic_scowl = scowl|hyphenated|contractions|all_names
pelic_scowl.remove('')
pelic_scowl = sorted(list(pelic_scowl))

with open('PELIC-SCOWL.txt', 'w') as f:
    for item in pelic_scowl:
        f.write("%s\n" % item)

In [18]:
# Removing hypenated words

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[0] not in hyphenated])

In [19]:
# Removing items that are only numbers or punctuation
# .isalpha() cannot be used without 'any' as this also removes hyphenated words

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if any(y.isalpha() for y in x[0])])

In [20]:
# Checking initial length of non_words list

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

680878
20381


In [21]:
# Removing items that are not purely alpha

non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[0].isalpha()])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26630
15775


In [23]:
# Removing proper names - NNP, NNPS

# non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[2] != 'NNP' and x[1] != 'NNPS'])
#print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
#print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

After manual checking, it was decided to keep in items tagged as NNP and NNPS as some items were in fact mistagged and were general capitalized nouns (NN) which were misspelled.

In [24]:
# Removing all words with length 1

non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if len(x[0]) > 1])

In [25]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26618
15766


In [26]:
# Removing all words with special characters (non_ascii)
# after checking, these are all foreign words with accents and other non-latin characters

# Creating function to check
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [27]:
non_word_list = set([x for y in non_words.misspelled_words.to_list() for x in y])
foreign_words = [x for x in non_word_list if is_ascii(x[0]) == False ]
foreign_words[:20]

[('opéra', 'opéra', 'NNP'),
 ('çark', 'çark', 'FW'),
 ('ãæ', 'ãæ', 'VBP'),
 ('currículm', 'currículm', 'NN'),
 ('íçãíãï', 'íçãíãï', 'NNP'),
 ('crêpe', 'crêpe', 'NN'),
 ('çáì', 'çáì', 'NNP'),
 ('çáçêõçá', 'çáçêõçá', 'NNP'),
 ('óáçãì', 'óáçãì', 'NNP'),
 ('niño', 'niño', 'NNP'),
 ('çáóäçá', 'çáóäçá', 'NNP'),
 ('ýçääì', 'ýçääì', 'NNP'),
 ('çáóíñçãíß', 'çáóíñçãíß', 'NNP'),
 ('áì', 'áì', 'NNP'),
 ('óæçá', 'óæçá', 'NNP'),
 ('çúøíêäí', 'çúøíêäí', 'NNP'),
 ('çáæìå', 'çáæìå', 'NNP'),
 ('íèþì', 'íèþì', 'NNP'),
 ('vinícius', 'vinícius', 'NNP'),
 ('æçáçóñé', 'æçáçóñé', 'NNP')]

In [28]:
# Removing foreign words
non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x not in foreign_words])

In [29]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

26295
15527


In [30]:
# Removing names from names list words
all_names = {y.lower() for y in all_names}
non_words.misspelled_words = non_words.misspelled_words.apply(lambda row: [x for x in row if x[0] not in all_names])

In [31]:
# Checking affect of removal

print(len([x for y in non_words.misspelled_words.to_list() for x in y]))
print(len(set([x for y in non_words.misspelled_words.to_list() for x in y])))

24526
14701


In [32]:
non_words.head()

,tok_lem_POS,misspelled_words,sentence
answer_id,,,
1,"[(i, i, PRP), (met, meet, VBD), (my, my, PRP$)...",[],[I met my friend Nife while I was studying in ...
2,"[(ten, ten, CD), (years, year, NNS), (ago, ago...",[],"[Ten years ago, I met a women on the train bet..."
3,"[(in, in, IN), (my, my, PRP$), (country, count...",[],"[In my country we usually don't use tea bags.,..."
4,"[(i, i, PRP), (organized, organize, VBD), (the...",[],[I organized the instructions by time.]
5,"[(first, first, RB), (,, ,, ,), (prepare, prep...",[],"[First, prepare a port, loose tea, and cup., S..."


Create new dataframe so that each misspelling token is a separate row.

In [33]:
# Removing rows with no misspellings

non_words2 = non_words.loc[non_words.misspelled_words.str.len() > 0,:].copy()

In [34]:
# Exploding the lists in misspelled words so that each misspelling gets its own row

non_words2 = non_words2.explode('misspelled_words')

In [35]:
# Keeping only the sentence containing the error (the first occurence of the error if repeated)

non_words2['sentence'] = list(zip([x[0] for x in non_words2.misspelled_words], non_words2.sentence))
non_words2['sentence'] = non_words2['sentence'].apply(
    lambda row: [i for i in row[1] if row[0] in lex.re_tokenize(i) or row[0]+"n't" in i.lower()])
non_words2['sentence'] = [x[0] for x in non_words2['sentence']]
non_words2 = non_words2.drop_duplicates(subset = ['misspelled_words','sentence'])

In [36]:
# Keeping the answer_id (which is no longer unique) as a separate column

non_words2 = non_words2.reset_index(drop = False)
non_words2.head()

,answer_id,tok_lem_POS,misspelled_words,sentence
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","I organized the instructions by time, beacause..."
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","next, you need to buy a box of tea in wallmart..."
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","First, you should take some hot water, you can..."
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","First, you should take some hot water, you can..."
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","Fitst, boil a water in a pot."


#### Adding a bigrams column, i.e. one token left and right of the misspelled word

In [37]:
# Creating a tokenized version of the sentence without punctuation and with the index for each token

non_words2['enumerated'] = non_words2.sentence.apply(lambda x: lex.re_tokenize(x)).apply(enumerate).apply(list)
non_words2.head()

,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","I organized the instructions by time, beacause...","[(0, i), (1, organized), (2, the), (3, instruc..."
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","next, you need to buy a box of tea in wallmart...","[(0, next), (1, you), (2, need), (3, to), (4, ..."
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","First, you should take some hot water, you can...","[(0, first), (1, you), (2, should), (3, take),..."
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","First, you should take some hot water, you can...","[(0, first), (1, you), (2, should), (3, take),..."
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","Fitst, boil a water in a pot.","[(0, fitst), (1, boil), (2, a), (3, water), (4..."


In [38]:
# Creating a function to extract the bigrams (1 word either side of misspelling)

def get_bigrams(misspelled_word, enumerated_list):
    if len(enumerated_list) <2:
        return []
    for tup in enumerated_list:
        if tup[1] == misspelled_word[0]:
            if tup[0] == 0:
                bigram = ' '.join([x[1] for x in (enumerated_list[tup[0]],enumerated_list[tup[0]+1])])
                return [(bigram,2)]
            if tup[0] == len(enumerated_list)-1:
                bigram = ' '.join([x[1] for x in (enumerated_list[tup[0]-1],enumerated_list[tup[0]])])
                return [(bigram,1)]
            else:
                bigram1 = ' '.join([x[1] for x in (enumerated_list[tup[0]-1],enumerated_list[tup[0]])])
                bigram2 = ' '.join([x[1] for x in (enumerated_list[tup[0]],enumerated_list[tup[0]+1])])
                return [(bigram1,1), (bigram2,2)]
            
# Notes:
# '1' and '2' added as identifiers to show which word in bigram is the misspelling
# Type 1 bigram = (word, misspelling), Type 2 bigram = (misspelling, word)

In [39]:
# Testing the function

test_list = non_words2.iloc[5,4]
print(test_list)

first_item = non_words2.iloc[5,1][0] # first item in list
middle_item = non_words2.iloc[5,1][4] # item in in middle of list
last_item = non_words2.iloc[5,1][8] # item at end of list
print('\n',first_item, middle_item, last_item)

[(0, 'every'), (1, 'paragragh'), (2, 's'), (3, 'instructions'), (4, 'depend'), (5, 'on'), (6, 'a'), (7, 'main'), (8, 'idea')]

 ('every', 'every', 'DT') ('depend', 'depend', 'VBP') ('idea', 'idea', 'NN')


In [40]:
#getbigrams

print(get_bigrams(first_item,test_list)) # One possible bigram
print(get_bigrams(middle_item,test_list)) # Two possible bigrams
print(get_bigrams(last_item,test_list)) # One possible bigram

[('every paragragh', 2)]
[('instructions depend', 1), ('depend on', 2)]
[('main idea', 1)]


In [41]:
# Applying getbigrams2

non_words2['bigrams'] = non_words2[['misspelled_words','enumerated']].apply(lambda x: get_bigrams(x[0],x[1]), axis=1)

In [42]:
non_words2.head()

,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated,bigrams
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","I organized the instructions by time, beacause...","[(0, i), (1, organized), (2, the), (3, instruc...","[(time beacause, 1), (beacause to, 2)]"
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","next, you need to buy a box of tea in wallmart...","[(0, next), (1, you), (2, need), (3, to), (4, ...","[(in wallmart, 1), (wallmart or, 2)]"
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","First, you should take some hot water, you can...","[(0, first), (1, you), (2, should), (3, take),...","[(use dovn, 1), (dovn mircowave, 2)]"
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","First, you should take some hot water, you can...","[(0, first), (1, you), (2, should), (3, take),...","[(dovn mircowave, 1), (mircowave or, 2)]"
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","Fitst, boil a water in a pot.","[(0, fitst), (1, boil), (2, a), (3, water), (4...","[(fitst boil, 2)]"


In [43]:
# The 14 misspellings with contracted words did not return any suggestions as the n't is handled differently by 
# the re_tokenize function which strips punctuation and the nltk tokenizer used to create the tok_lem_POS column

no_bigrams = non_words2.loc[(non_words2['bigrams'].isnull()) & (non_words2.enumerated.str.len() >1),:]
print(len(no_bigrams))
no_bigrams.head(20)

14


,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated,bigrams
870,1469,"[(about, about, IN), (4, 4, CD), (days, day, N...","(shoudl, shoudl, VBP)","Anyway, we went to a hospital and the doctor s...","[(0, anyway), (1, we), (2, went), (3, to), (4,...",None
1055,1797,"[(i, i, PRP), (failed, fail, VBD), (the, the, ...","(idid, idid, NNP)",I failed the test because Ididn't study.,"[(0, i), (1, failed), (2, the), (3, test), (4,...",None
2333,4804,"[(bill, bill, NNP), (shoul, shoul, VBP), (n't,...","(shoul, shoul, VBP)",Bill shouln't have lied about the price of the...,"[(0, bill), (1, shouln), (2, t), (3, have), (4...",None
2336,4830,"[(caroline, caroline, NNP), (must, must, MD), ...","(sould, sould, VBP)",Caroline souldn't have insulted Bill in front ...,"[(0, caroline), (1, souldn), (2, t), (3, have)...",None
2598,6346,"[(last, last, JJ), (year, year, NN), (,, ,, ,)...","(sould, sould, MD)",I told the officer that he caused the accident...,"[(0, i), (1, told), (2, the), (3, officer), (4...",None
2601,6346,"[(last, last, JJ), (year, year, NN), (,, ,, ,)...","(sould, sould, VBP)",I told the officer that he caused the accident...,"[(0, i), (1, told), (2, the), (3, officer), (4...",None
3584,8640,"[(before, before, IN), (i, i, PRP), (came, com...","(daoe, daoe, VBP)","I like to play soccer,but in Korea daoen't hav...","[(0, i), (1, like), (2, to), (3, play), (4, so...",None
3862,10001,"[(you, you, PRP), (shoud, shoud, VBP), (n't, n...","(shoud, shoud, VBP)",You shoudn't,"[(0, you), (1, shoudn), (2, t)]",None
9721,23939,"[(i, i, PRP), (have, have, VBP), (an, a, DT), ...","(sould, sould, VBP)",I souldn't have gone to his wedding party.,"[(0, i), (1, souldn), (2, t), (3, have), (4, g...",None
10736,26126,"[(i, i, PRP), (like, like, VBP), (cooking, coo...","(doed, doed, VBP)",But In pittsburgh doedn't have good traditiona...,"[(0, but), (1, in), (2, pittsburgh), (3, doedn...",None


In [44]:
# Creating function to fix these items

misspelled_contractions = [x[0] for x in no_bigrams.misspelled_words]

def fix_contractions(word):
    if word == "t":
        word = "n't"
    if word[:-1] in misspelled_contractions:
        word = word[:-1]
    return word

In [45]:
# Applying the above function to the selected rows

mask = non_words2.index.isin(no_bigrams.index)
non_words2.loc[mask, 'enumerated'] = non_words2.loc[mask, 'enumerated'].apply(
    lambda row: [(x[0],fix_contractions(x[1])) for x in row])

In [46]:
# Reapplying the bigrams function

non_words2['bigrams'] = non_words2[['misspelled_words','enumerated']].apply(lambda x: get_bigrams(x[0],x[1]), axis=1)

In [47]:
# Re-checking that all rows have bigrams
non_words2.loc[(non_words2['bigrams'].isnull()) & (non_words2.enumerated.str.len() >1),:]

,answer_id,tok_lem_POS,misspelled_words,sentence,enumerated,bigrams


In [48]:
# Deleting the enumerated column as no longer necessary

del non_words2['enumerated']

In [49]:
# Renaming the 'misspelled_words' column as there is only one word in each row

non_words2 = non_words2.rename(columns={"misspelled_words": "misspelling"})

In [50]:
# Checking final non_words2 dataframe

non_words2.head()

,answer_id,tok_lem_POS,misspelling,sentence,bigrams
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","I organized the instructions by time, beacause...","[(time beacause, 1), (beacause to, 2)]"
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","next, you need to buy a box of tea in wallmart...","[(in wallmart, 1), (wallmart or, 2)]"
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","First, you should take some hot water, you can...","[(use dovn, 1), (dovn mircowave, 2)]"
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","First, you should take some hot water, you can...","[(dovn mircowave, 1), (mircowave or, 2)]"
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","Fitst, boil a water in a pot.","[(fitst boil, 2)]"


In [51]:
# Total number of non-words (tokens)
print(len(non_words2))

# Total number of non-words (types)
print(non_words2.misspelling.nunique())

19784
14701


#### Creating a dataframe of misspellings
In the `non-words2` dataframe above, each row is an occurrence of a misspelling (i.e. _tokens_ ). We also want a dataframe where each row is a misspelling _type_ with frequency information attached.

In [52]:
# Gathering the total misspellings and bigrams

total_unigram_misspell = [x for x in non_words2['misspelling']]
total_bigram_misspell = [x[0] for y in non_words2['bigrams'] for x in y] #flattened list

In [53]:
print(total_unigram_misspell[:5])
print(total_bigram_misspell[:5])

[('beacause', 'beacause', 'NN'), ('wallmart', 'wallmart', 'NN'), ('dovn', 'dovn', 'NN'), ('mircowave', 'mircowave', 'VBP'), ('fitst', 'fitst', 'NNP')]
['time beacause', 'beacause to', 'in wallmart', 'wallmart or', 'use dovn']


In [54]:
# Creating frequency dictionaries for unigrams and bigrams

unigram_misspell_freq_dict = {}
for word in total_unigram_misspell:
    if word not in unigram_misspell_freq_dict:
        unigram_misspell_freq_dict[word] = 1
    else:
        unigram_misspell_freq_dict[word] += 1

In [55]:
bigram_misspell_freq_dict = {}
for bigram in total_bigram_misspell:
    if bigram not in bigram_misspell_freq_dict:
        bigram_misspell_freq_dict[bigram] = 1
    else:
        bigram_misspell_freq_dict[bigram] += 1

In [56]:
# Checking dictionaries

print(random.sample(list(unigram_misspell_freq_dict),5))
print(random.sample(list(bigram_misspell_freq_dict),5))

[('pullution', 'pullution', 'NN'), ('poors', 'poors', 'NNS'), ('iss', 'iss', 'NNP'), ('diffrints', 'diffrints', 'NNS'), ('experense', 'experense', 'VBP')]
['a mixtire', 'statistics nchs', 'any relarionship', 'books iwhen', 'spcial person']


In [57]:
# Remove duplicates

final_unigram_misspellings = sorted(list(set(total_unigram_misspell)))
final_bigram_misspellings = sorted(list(set(total_bigram_misspell)))
print(len(final_unigram_misspellings))
print(len(final_bigram_misspellings))

14701
30188


In [58]:
# Constructing misspell_df

misspell_df = pd.DataFrame(final_unigram_misspellings)
misspell_df.head()

,0,1,2
0,aa,aa,NNP
1,aa,aa,VB
2,aabout,aabout,IN
3,aad,aad,JJ
4,aain,aain,VBP


In [59]:
# Renaming columns to match other DataFrames in this notebook

misspell_df.rename(columns = {0: 'misspelling',1:'lemma',2:'POS'}, inplace = True)

In [60]:
# Recreating tok_lem_POS column to match dictionary

misspell_df['tok_lem_POS'] = list(zip(misspell_df.misspelling, misspell_df.lemma, misspell_df.POS))
misspell_df.head()

,misspelling,lemma,POS,tok_lem_POS
0,aa,aa,NNP,"(aa, aa, NNP)"
1,aa,aa,VB,"(aa, aa, VB)"
2,aabout,aabout,IN,"(aabout, aabout, IN)"
3,aad,aad,JJ,"(aad, aad, JJ)"
4,aain,aain,VBP,"(aain, aain, VBP)"


In [61]:
# Mapping dictionary to DataFrame

misspell_df['freq'] = misspell_df['tok_lem_POS'].map(unigram_misspell_freq_dict)

In [62]:
# Sorting by frequency

misspell_df = misspell_df.sort_values(by=['freq'], ascending=False)

In [63]:
# Resetting index and deleting unnecesary columns

misspell_df = misspell_df.reset_index(drop = True)
del misspell_df['lemma']
del misspell_df['POS']

misspell_df.head(20)

,misspelling,tok_lem_POS,freq
0,alot,"(alot, alot, NN)",103
1,studing,"(studing, studing, VBG)",62
2,tofel,"(tofel, tofel, NNP)",39
3,goverment,"(goverment, goverment, NN)",36
4,iam,"(iam, iam, NNP)",28
5,finaly,"(finaly, finaly, NNP)",26
6,beatiful,"(beatiful, beatiful, JJ)",24
7,becuase,"(becuase, becuase, NN)",24
8,oss,"(oss, oss, NNP)",22
9,differents,"(differents, differents, NNS)",21


#### scowl_supp
The following is the basis for the 'scowl_supp' list used earlier. Here, errors with a frequency of 10 or more were manually checked, and if determined to be a real word, were added to the scowl_supp list. There were originally 267 items which met this criteria.

In [64]:
print(len(misspell_df.loc[misspell_df.freq >= 10]))
misspell_df.loc[misspell_df.freq >= 10]

58


,misspelling,tok_lem_POS,freq
0,alot,"(alot, alot, NN)",103
1,studing,"(studing, studing, VBG)",62
2,tofel,"(tofel, tofel, NNP)",39
3,goverment,"(goverment, goverment, NN)",36
4,iam,"(iam, iam, NNP)",28
5,finaly,"(finaly, finaly, NNP)",26
6,beatiful,"(beatiful, beatiful, JJ)",24
7,becuase,"(becuase, becuase, NN)",24
8,oss,"(oss, oss, NNP)",22
9,differents,"(differents, differents, NNS)",21


### Possible segmentation

Selected segmenter and spellchecker: SymSpell https://github.com/mammothb/symspellpy

There is a dictionary file which which needs to be installed (saved to repo):
[frequency_dictionary_en_82_765.txt](https://symspellpy.readthedocs.io/en/latest/users/installing.html)

To install symspellpy the first time, use pip in command line: `pip install -U symspellpy`

Prior to spelling correct, we first consider using the segmenter. This is a potentially useful first step as misspellings like 'alot' or 'dogmeat' will be separated into 'a lot' and 'dog meat' rather than corrected to a single word like 'lot'.  

However, when segementing misspellings, the segmenter over performs, segmenting non-words into real words where it was clearly not intended, e.g. _improtant_ into _imp rot ant_ or _befor_ into _be for_. As such, the segmenting will not be automated. 

Instead, we rely on the bigram spelling correction which correctly segments items like _alot, iam, everytime,_ etc.

In [65]:
# Setting up symspell

from itertools import islice
import pkg_resources
from symspellpy import SymSpell
from symspellpy import Verbosity
sym_spell = SymSpell()
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, 0, 1)

# Print out first 5 elements to demonstrate that dictionary is successfully loaded
list(islice(sym_spell.words.items(), 5))

[('the', 23135851162),
 ('of', 13151942776),
 ('and', 12997637966),
 ('to', 12136980858),
 ('a', 9081174698)]

In [66]:
# Testing segmenter with 'alot' and 'dogmeat'

# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# It is also possible to display frequency with result.distance_sum and edit distance with .log_prob_sum

True

In [67]:
# Creating function for applying the above code

def get_segments(word):
    segments = sym_spell.word_segmentation(word)
    if len(segments.corrected_string.split(' ')) > 1 \
    and segments.corrected_string.split(' ')[0] in scowl and segments.corrected_string.split(' ')[1] in scowl:
        return segments.corrected_string
    else:
        return word

In [68]:
# Testing function

print(get_segments('dogmeat')) # Should be segmented
print(get_segments('fireplace')) # Should not be segmented
print(get_segments('becuase')) # Should not be segmented

dog meat
fireplace
becuase


In [69]:
# Applying the function to create a new column

misspell_df['segments'] =  misspell_df['misspelling'].apply(get_segments)
misspell_df.head(10)

,misspelling,tok_lem_POS,freq,segments
0,alot,"(alot, alot, NN)",103,a lot
1,studing,"(studing, studing, VBG)",62,stu ding
2,tofel,"(tofel, tofel, NNP)",39,tofel
3,goverment,"(goverment, goverment, NN)",36,g over men t
4,iam,"(iam, iam, NNP)",28,i am
5,finaly,"(finaly, finaly, NNP)",26,final y
6,beatiful,"(beatiful, beatiful, JJ)",24,beat if ul
7,becuase,"(becuase, becuase, NN)",24,becuase
8,oss,"(oss, oss, NNP)",22,oss
9,differents,"(differents, differents, NNS)",21,different s


In [70]:
# Deleting this new column as segmentation creates false segments of misspelled words

del misspell_df['segments']

### Applying spelling correction

In some ways SymSpell is not ideal as full sentence context is not considered, only general frequencies. However, other well-known spellcheckers (hunspell, pyspell, etc.) use the same strategy - frequency based criteria for suggestions, without considering immediate cotext. As such, we have followed this common practice, but it is important to remember that accuracy of corrected tokens will not be 100% and must be taken into consideration.

As a compromise and to consider context, spelling corrections based on bigrams is first implemented. If no suggestions are available, spelling corrections based on unigrams are implemented.

In [71]:
# Testing spelling suggestions with 'becuase'

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

input_term = "becuase"
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2, #Edit distance can be adjusted
                               transfer_casing=True, #Optional argument set to ignore case
                              include_unknown=True) #Return same word if unknown
for suggestion in suggestions:
    print(suggestion)  

because, 1, 271323986


In [72]:
# Creating function for finding unigram suggestions

def get_unigram_suggestions(word):
    if len(word) >= 4:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST,max_edit_distance=2, transfer_casing=True)
    else:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST,max_edit_distance=1, transfer_casing=True)
    return [str(x).split(',') for x in suggestions]

In [73]:
# Testing function

get_unigram_suggestions('becuase')

[['because', ' 1', ' 271323986']]

**Note**: The function has a variable edit distance: words of length 4 or more get edit distance of 2, shorter words get edit distance of 1. These preferences can be adjusted in the function if desired.

In [74]:
# Testing spelling suggestions with 'becuase of'

max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
if not sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1):
    print("Dictionary file not found")
if not sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2):
    print("Bigram dictionary file not found")
input_term = 'becuase of'
max_edit_distance_lookup = 2
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance_lookup)
for suggestion in suggestions:
    print(suggestion) 

because of, 1, 3481714


In [75]:
# Creating function for finding bigram suggestions

def get_bigram_suggestions(bigram):
    suggestions = sym_spell.lookup_compound(bigram, max_edit_distance_lookup)
    for suggestion in suggestions:
        return [str(x).split(',') for x in suggestions] 

In [76]:
# Testing function
get_bigram_suggestions('worq harg')

[['work hard', ' 2', ' 53229']]

In [77]:
# Returing to non_words2 dataframe and applying functions to create new column

# Creating unigram suggestions column

non_words2['unigram_suggestions'] =  non_words2['misspelling'].apply(
    lambda x: get_unigram_suggestions(x[0]))

In [78]:
# Turning into tuples for easier processing

non_words2.unigram_suggestions = non_words2.unigram_suggestions.apply(
    lambda row: [tuple(x) for x in row])

In [79]:
# Creating bigram suggestions column

non_words2['bigram_suggestions'] =  non_words2['bigrams'].apply(
    lambda row: [(get_bigram_suggestions(x[0]),x[1]) for x in row])

In [80]:
# Flattening and turning into tuples for easier processing

non_words2['bigram_suggestions'] = non_words2.bigram_suggestions.apply(
    lambda row: [(tuple([i for j in x[0] for i in j]),x[1]) for x in row])

In [81]:
# Checking how many items without suggestions

print(len(non_words2.loc[(non_words2.unigram_suggestions.str.len() == 0),:]))
print(len(non_words2.loc[(non_words2.bigram_suggestions.str.len() == 0),:]))
print(len(non_words2.loc[(non_words2.bigram_suggestions.str.len() == 0) & (non_words2.unigram_suggestions.str.len() == 0),:]))

1896
114
15


In [82]:
non_words2.loc[(non_words2.bigram_suggestions.str.len() == 0) & (non_words2.unigram_suggestions.str.len() == 0),:]

,answer_id,tok_lem_POS,misspelling,sentence,bigrams,unigram_suggestions,bigram_suggestions
71,105,"[(here, here, RB), (are, be, VBP), (a, a, DT),...","(asdfkjdlkfjadlfjalsdf, asdfkjdlkfjadlfjalsdf,...",asdfkjdlkfjadlfjalsdf,[],[],[]
3913,10490,"[(getting, get, VBG), (good, good, NNP), (grad...","(quintcareers, quintcareers, NNS)",Quintcareers.,[],[],[]
6822,18380,"[(hi, hi, NNP), (,, ,, ,), (adrian, adrian, NN...","(yeonjea, yeonjea, NN)",- Yeonjea.,[],[],[]
7838,20089,"[(uuuuuuu, uuuuuuu, NN)]","(uuuuuuu, uuuuuuu, NN)",uuuuuuu,[],[],[]
7839,20096,"[(yhryr, yhryr, NN)]","(yhryr, yhryr, NN)",yhryr,[],[],[]
8459,21585,"[(1, 1, CD), (., ., .), (individual, individua...","(shadizubeidi, shadizubeidi, NNP)",Shadizubeidi.,[],[],[]
8621,21845,"[(when, when, WRB), (i, i, PRP), ('m, be, VBP)...","(jaaaaaaaaaaaaaaaaaaajajjajajajaja, jaaaaaaaaa...",Jaaaaaaaaaaaaaaaaaaajajjajajajaja ;),[],[],[]
9831,24136,"[(hsthethghryjyjy, hsthethghryjyjy, NN)]","(hsthethghryjyjy, hsthethghryjyjy, NN)",hsthethghryjyjy,[],[],[]
10637,25988,"[(fibromyalgia, fibromyalgia, NNP), (?, ?, .),...","(fibromyalgia, fibromyalgia, NNP)",Fibromyalgia ?,[],[],[]
12460,30421,"[(stayhealthy, stayhealthy, NN)]","(stayhealthy, stayhealthy, NN)",StayHealthy,[],[],[]


Items with no suggestions - these will be left in their original form though manual corrections could be applied if desired.

Next, we create a new column with just the most likely correction (based on frequency). Bigram suggestions are given preference before unigram suggestions. If there is no suggestion, the original word is returned.

In [83]:
# Create new column with just the most likely correction (based on frequency)

def sort_unigram_tuple(tup):  
    tup.sort(key = lambda x: x[2], reverse=True)  
    return tup    

In [84]:
# Keeping the unigram correction with the highest frequency

non_words2['unigram_correction'] = [sort_unigram_tuple(x)[0][0] if len(x) != 0 else np.NaN for x in non_words2['unigram_suggestions']]

In [85]:
# Create new column with just the most likely correction (based on frequency)

def sort_bigram_tuple(tup):  
    tup.sort(key = lambda x: x[0][2], reverse=True)  
    return tup    

In [86]:
# Keeping the bigram correction with the highest frequency

non_words2['bigram_correction'] = [(sort_bigram_tuple(x)[0][0][0],x[0][1]) if len(x) != 0 else np.NaN for x in non_words2['bigram_suggestions']]

In [87]:
# Splitting the bigram corrections into separate words

mask = non_words2.loc[non_words2.bigram_correction.isnull() == False].index
non_words2.loc[mask, 'bigram_correction'] = non_words2.loc[mask, 'bigram_correction'].apply(
    lambda x: (x[0].split(),x[1]))

Some bigrams that were previously two words have now been corrected to one word, e.g. _paragragh, s --> paragraphs._  
These will now be labelled Type 3.

In [88]:
mask2 = non_words2.loc[mask].loc[non_words2.loc[mask].bigram_correction.apply(lambda x: len(x[0])==1)].index 
non_words2.loc[mask2].head()

#142 Type 3 items

,answer_id,tok_lem_POS,misspelling,sentence,bigrams,unigram_suggestions,bigram_suggestions,unigram_correction,bigram_correction
5,16,"[(every, every, DT), (paragragh, paragragh, NN...","(paragragh, paragragh, NN)",Every paragragh's instructions depend on a mai...,"[(every paragragh, 1), (paragragh s, 2)]","[(paragraph, 1, 24855495)]","[((paragraphs, 2, 5579470), 2), ((every para...",paragraph,"([paragraphs], 2)"
142,200,"[(hi, hi, NNP), (,, ,, ,), (i, i, PRP), ('m, b...","(jotard, jotard, NNP)","Hi, I'm Jotard.","[(m jotard, 1)]","[(jotted, 2, 66054), (retard, 2, 643417), ...","[((mustard, 3, 2820441), 1)]",jotted,"([mustard], 1)"
290,378,"[(my, my, PRP$), (writing, write, VBG), (teach...","(teachet, teachet, JJ)",My writing teachet`s name is ANON_NAME_0.,"[(writing teachet, 1), (teachet s, 2)]","[(teaches, 1, 5599377), (teacher, 1, 48002...","[((teachers, 2, 49832283), 2), ((writing tea...",teaches,"([teachers], 2)"
315,444,"[(topic, topic, NNP), (sentence, sentence, NN)...","(gaint, gaint, NNP)","For example, the Rose's Gaint Boutique and Hea...","[(s gaint, 1), (gaint boutique, 2)]","[(gait, 1, 822923), (saint, 1, 38929141), ...","[((saint, 2, 38929141), 1), ((saint boutique...",gait,"([saint], 1)"
353,542,"[(topic, topic, NNP), (sentence, sentence, NN)...","(gaint, gaint, NNP)","For example, the Rose's Gaint Boutique and Hea...","[(s gaint, 1), (gaint boutique, 2)]","[(gait, 1, 822923), (saint, 1, 38929141), ...","[((saint, 2, 38929141), 1), ((saint boutique...",gait,"([saint], 1)"


In [89]:
non_words2.loc[mask2,'bigram_correction'] = non_words2.loc[mask2,'bigram_correction'].apply(lambda x: (x[0],3))

In [90]:
# Keeping only the word in the bigram correction that was originally misspelled

def corrected_only(bigram_tuple):
    bigram_list = bigram_tuple[0]
    bigram_type = bigram_tuple[1]
    for tup in bigram_tuple:
        if bigram_type == 1:
            corrected_word = bigram_list[1:]
        if bigram_type == 2:
            corrected_word = bigram_list[:-1]
        if bigram_type == 3:
            corrected_word = bigram_list
    return ' '.join(corrected_word)

# Type 1 bigram = (word, misspelling), Type 2 bigram = (misspelling, word), Type 3 bigram = misspelling only

In [91]:
# Applying the function

non_words2.loc[mask, 'bigram_correction'] = non_words2.loc[mask, 'bigram_correction'].apply(
    lambda x: corrected_only(x))

In [92]:
# Creating a 'final_correction' column with order of preference: bigram_correction, unigram_correction, misspelling

# Changing empty strings to NaN in the bigram correction column
non_words2.bigram_correction.replace("", np.nan, inplace=True)

# Creating a tuple of these three items
non_words2['final_correction'] = list(zip(non_words2.misspelling.apply(
    lambda x: x[1]), non_words2.unigram_correction, non_words2.bigram_correction))

# Choosing which item based on if strings or not (i.e. NaNs)
non_words2['final_correction'] = [x[2] if isinstance(x[2], str) else x[1] if isinstance(x[1], str) else x[0] for x in non_words2['final_correction']]

In [93]:
# Create correction_POS column

non_words2['final_correction_POS'] = list(zip(non_words2.final_correction, non_words2.final_correction, non_words2.misspelling.apply(lambda x: x[2])))
non_words2.head(6)

,answer_id,tok_lem_POS,misspelling,sentence,bigrams,unigram_suggestions,bigram_suggestions,unigram_correction,bigram_correction,final_correction,final_correction_POS
0,8,"[(i, i, PRP), (organized, organize, VBD), (the...","(beacause, beacause, NN)","I organized the instructions by time, beacause...","[(time beacause, 1), (beacause to, 2)]","[(because, 1, 271323986)]","[((because to, 1, 3213023), 2), ((time becau...",because,because,because,"(because, because, NN)"
1,11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","(wallmart, wallmart, NN)","next, you need to buy a box of tea in wallmart...","[(in wallmart, 1), (wallmart or, 2)]","[(walmart, 1, 2269839)]","[((in wall art, 1, 99805), 1), ((wall art or...",walmart,wall art,wall art,"(wall art, wall art, NN)"
2,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(dovn, dovn, NN)","First, you should take some hot water, you can...","[(use dovn, 1), (dovn mircowave, 2)]","[(dove, 1, 3253560), (donn, 1, 299470), (d...","[((down microwave, 2, 1960), 2), ((use down,...",dove,down,down,"(down, down, NN)"
3,13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","(mircowave, mircowave, VBP)","First, you should take some hot water, you can...","[(dovn mircowave, 1), (mircowave or, 2)]","[(microwave, 1, 8934594)]","[((microwave or, 1, 22584), 2), ((down micro...",microwave,microwave,microwave,"(microwave, microwave, VBP)"
4,15,"[(in, in, IN), (my, my, PRP$), (country, count...","(fitst, fitst, NNP)","Fitst, boil a water in a pot.","[(fitst boil, 2)]","[(fist, 1, 7319405), (first, 1, 578161543)...","[((first boil, 1, 1366), 2)]",fist,first,first,"(first, first, NNP)"
5,16,"[(every, every, DT), (paragragh, paragragh, NN...","(paragragh, paragragh, NN)",Every paragragh's instructions depend on a mai...,"[(every paragragh, 1), (paragragh s, 2)]","[(paragraph, 1, 24855495)]","[((paragraphs, 2, 5579470), 2), ((every para...",paragraph,paragraphs,paragraphs,"(paragraphs, paragraphs, NN)"


### Incorporating corrections into `pelic_df`

In [94]:
# Focusing on rows in pelif_df containing spelling mistakes to replace

mask = pelic_df.index.isin(non_words2.answer_id)
print(len(pelic_df))
print(len(pelic_df.loc[mask]))

46230
9695


In [95]:
# Transforming non_words2 back so that each row is a text (with a list of the misspellings and final corrections)

# First create a column with tuples of misspellings and their corrections
non_words2['misspelling_correction'] = list(zip(non_words2.misspelling, non_words2.final_correction_POS))

In [96]:
# Then group by the text and combine the misspelling_corrections
non_words3 = non_words2[['answer_id','tok_lem_POS','misspelling_correction']]
non_words3 = non_words2.groupby('answer_id').agg({'tok_lem_POS':'first','misspelling_correction': sum}).reset_index()
non_words3.misspelling_correction = non_words3.misspelling_correction.apply(lambda x:list(zip(x[::2], x[1::2]))) #combining items into tuples
non_words3 = non_words3.set_index('answer_id') # use answer_id as new index since it is now unique
non_words3.head()

,tok_lem_POS,misspelling_correction
answer_id,,
8,"[(i, i, PRP), (organized, organize, VBD), (the...","[((beacause, beacause, NN), (because, because,..."
11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","[((wallmart, wallmart, NN), (wall art, wall ar..."
13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","[((dovn, dovn, NN), (down, down, NN)), ((mirco..."
15,"[(in, in, IN), (my, my, PRP$), (country, count...","[((fitst, fitst, NNP), (first, first, NNP))]"
16,"[(every, every, DT), (paragragh, paragragh, NN...","[((paragragh, paragragh, NN), (paragraphs, par..."


In [97]:
# Creating a function to find and replace misspellings

def replace_misspelling(tok_lem_POS, misspelling_correction):
    tok_lem_POS_corrected = []
    misspellings = [x[0] for x in misspelling_correction]
    corrections = [x[1] for x in misspelling_correction]
    correction_dict = dict((x, y) for x, y in misspelling_correction)
    for tok in tok_lem_POS:
        if tok in misspellings:
            tok = correction_dict[tok]
        tok_lem_POS_corrected.append(tok)
    return tok_lem_POS_corrected

In [98]:
# Applying the above function

non_words3['tok_lem_POS_corrected'] = non_words3[['tok_lem_POS','misspelling_correction']].apply(
    lambda x: replace_misspelling(x[0],x[1]), axis=1)

non_words3.head()

,tok_lem_POS,misspelling_correction,tok_lem_POS_corrected
answer_id,,,
8,"[(i, i, PRP), (organized, organize, VBD), (the...","[((beacause, beacause, NN), (because, because,...","[(i, i, PRP), (organized, organize, VBD), (the..."
11,"[(to, to, TO), (make, make, VB), (tea, tea, NN...","[((wallmart, wallmart, NN), (wall art, wall ar...","[(to, to, TO), (make, make, VB), (tea, tea, NN..."
13,"[(first, first, RB), (,, ,, ,), (you, you, PRP...","[((dovn, dovn, NN), (down, down, NN)), ((mirco...","[(first, first, RB), (,, ,, ,), (you, you, PRP..."
15,"[(in, in, IN), (my, my, PRP$), (country, count...","[((fitst, fitst, NNP), (first, first, NNP))]","[(in, in, IN), (my, my, PRP$), (country, count..."
16,"[(every, every, DT), (paragragh, paragragh, NN...","[((paragragh, paragragh, NN), (paragraphs, par...","[(every, every, DT), (paragraphs, paragraphs, ..."


In [99]:
# Creating a dictionary of answer_id and tok_lem_POS_corrected

corrected_text_dict = pd.Series(non_words3.tok_lem_POS_corrected.values,non_words3.index).to_dict()

In [100]:
# Adding the tok_lem_POS_corrected column to pelic_df

pelic_df['tok_lem_POS_corrected'] = pelic_df.index.map(corrected_text_dict)

Texts that have not had any corrections are labelled _NaN_ in the tok_lem_POS_corrected column. However, if you want to replace these with the orignal texts, you can use the following code instead of the one in the previous cell:  

```
pelic_df['tok_lem_POS_corrected'] = pelic_df.index.map(corrected_text_dict).fillna(pelic_df['tok_lem_POS'])
```

In [101]:
# Checking with 'becuase'

print(pelic_df.loc[pelic_df.text.str.contains('becuase')].iloc[1,11]) #uncorrected
print(pelic_df.loc[pelic_df.text.str.contains('becuase')].iloc[1,12]) #corrected

(('My', 'my', 'PRP$'), ('friend', 'friend', 'NN'), ('is', 'be', 'VBZ'), ('realy', 'realy', 'JJ'), ('nise', 'nise', 'RB'), ('guy', 'guy', 'NN'), ('.', '.', '.'), ('I', 'i', 'PRP'), ('like', 'like', 'VBP'), ('hem', 'hem', 'JJ'), ('becuase', 'becuase', 'NN'), ('he', 'he', 'PRP'), ('is', 'be', 'VBZ'), ('friendlly', 'friendlly', 'RB'), ('and', 'and', 'CC'), ('lovliy', 'lovliy', 'NN'), ('.', '.', '.'))
[('my', 'my', 'PRP$'), ('friend', 'friend', 'NN'), ('is', 'be', 'VBZ'), ('real', 'real', 'JJ'), ('nice', 'nice', 'RB'), ('guy', 'guy', 'NN'), ('.', '.', '.'), ('i', 'i', 'PRP'), ('like', 'like', 'VBP'), ('hem', 'hem', 'JJ'), ('because', 'because', 'NN'), ('he', 'he', 'PRP'), ('is', 'be', 'VBZ'), ('friendly', 'friendly', 'RB'), ('and', 'and', 'CC'), ('lovely', 'lovely', 'NN'), ('.', '.', '.')]


We can see here that many appropriate corrections have been made, including _becuase_ -> _because_ , _nise_ -> _nice_ , _friendlly_ -> _friendly_ and _lovily_ -> _lovely_ .  
Importantly, incorrect spellings that are actual words, e.g. _hem_ (should be _him_ in this case) are not corrected. In addition, as limited context is considered, there will be some inaccuracies, e.g. _realy_ (marked as an adj) -> _real_ rather than _really_ (_real nice_ is a frequent bigram).

In [105]:
pelic_df.head(10)

,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text_len,text,tokens,tok_lem_POS,tok_lem_POS_corrected
answer_id,,,,,,,,,,,,,
1,eq0,Arabic,Male,149,4,g,5,1,177,I met my friend Nife while I was studying in a...,"[I, met, my, friend, Nife, while, I, was, stud...","((I, i, PRP), (met, meet, VBD), (my, my, PRP$)...",NaN
2,am8,Thai,Female,149,4,g,5,1,137,"Ten years ago, I met a women on the train betw...","[Ten, years, ago, ,, I, met, a, women, on, the...","((Ten, ten, CD), (years, year, NNS), (ago, ago...",NaN
3,dk5,Turkish,Female,115,4,w,12,1,64,In my country we usually don't use tea bags. F...,"[In, my, country, we, usually, do, n't, use, t...","((In, in, IN), (my, my, PRP$), (country, count...",NaN
4,dk5,Turkish,Female,115,4,w,13,1,6,I organized the instructions by time.,"[I, organized, the, instructions, by, time, .]","((I, i, PRP), (organized, organize, VBD), (the...",NaN
5,ad1,Korean,Female,115,4,w,12,1,59,"First, prepare a port, loose tea, and cup.\nSe...","[First, ,, prepare, a, port, ,, loose, tea, ,,...","((First, first, RB), (,, ,, ,), (prepare, prep...",NaN
6,ad1,Korean,Female,115,4,w,13,1,2,By time,"[By, time]","((By, by, IN), (time, time, NN))",NaN
7,eg5,Korean,Female,115,4,w,12,1,39,"First, prepare your cup, loose tea or bag tea,...","[First, ,, prepare, your, cup, ,, loose, tea, ...","((First, first, RB), (,, ,, ,), (prepare, prep...",NaN
8,eg5,Korean,Female,115,4,w,13,1,35,"I organized the instructions by time, beacause...","[I, organized, the, instructions, by, time, ,,...","((I, i, PRP), (organized, organize, VBD), (the...","[(i, i, PRP), (organized, organize, VBD), (the..."
9,ad1,Korean,Female,115,4,w,12,2,59,"First, prepare a port, loose tea, and cup.\nSe...","[First, ,, prepare, a, port, ,, loose, tea, ,,...","((First, first, RB), (,, ,, ,), (prepare, prep...",NaN


In [103]:
# Write out new PELIC_compiled.csv

pelic_df.to_csv('PELIC_compiled_spellcorrected.csv', encoding='utf-8', index=True)

In [104]:
# Pickle new pelic_df dataframe

pelic_df.to_pickle('pelic_spellcorrected.pkl')

If preferred, this entire spelling correctin process can also be applied to [`answer.csv`]() instead of `PELIC_compiled`.

[Back to top](#Corrected-spelling)